3. 

In [1]:
import pandas as pd
import numpy as np

In [2]:
users = pd.read_csv('users.dat', sep='::',engine ='python',header=None )
list1 = ['UserID', 'Gender', 'Age', 'Occupation', 'Zip-code']
users.columns = list1

movies = pd.read_csv('movies.dat', sep='::',engine ='python',header=None)
list3 = ['MovieID', 'Title', 'Genres']
movies.columns = list3
movies['name'] = movies['Title'].str.slice(start=0,stop=-7)
movies['year'] = movies['Title'].str[-5:-1]     
movies['year'] = movies['year'].astype(int)
del movies['Title']

ratings = pd.read_csv('ratings.dat', sep='::',engine ='python',header=None)
list2 = ['UserID', 'MovieID', 'Rating', 'Timestamp']
ratings.columns = list2

a)

In [3]:
df1 = pd.merge(pd.merge(ratings, users), movies)
df1[:5]

,UserID,MovieID,Rating,Timestamp,Gender,Age,Occupation,Zip-code,Genres,name,year
0,1,1193,5,978300760,F,1,10,48067,Drama,One Flew Over the Cuckoo's Nest,1975
1,2,1193,5,978298413,M,56,16,70072,Drama,One Flew Over the Cuckoo's Nest,1975
2,12,1193,4,978220179,M,25,12,32793,Drama,One Flew Over the Cuckoo's Nest,1975
3,15,1193,4,978199279,M,25,7,22903,Drama,One Flew Over the Cuckoo's Nest,1975
4,17,1193,5,978158471,M,50,1,95350,Drama,One Flew Over the Cuckoo's Nest,1975


In [4]:
table1 = df1.pivot_table('Rating',\
index=['name','year'],aggfunc='count')\
.sort_values(by='Rating',ascending = False).head(5)
table1

,,Rating
name,year,
American Beauty,1999,3428
Star Wars: Episode IV - A New Hope,1977,2991
Star Wars: Episode V - The Empire Strikes Back,1980,2990
Star Wars: Episode VI - Return of the Jedi,1983,2883
Jurassic Park,1993,2672


b)

In [5]:
df1.dtypes

UserID         int64
MovieID        int64
Rating         int64
Timestamp      int64
Gender        object
Age            int64
Occupation     int64
Zip-code      object
Genres        object
name          object
year           int64
dtype: object

In [6]:
df2 = df1[(df1.Gender=='F')&(df1.year>=1995)\
&(df1.year<=2000)]
df2[:5]

,UserID,MovieID,Rating,Timestamp,Gender,Age,Occupation,Zip-code,Genres,name,year
1725,1,661,3,978302109,F,1,10,48067,Animation|Children's|Musical,James and the Giant Peach,1996
1731,59,661,3,977933844,F,50,1,55413,Animation|Children's|Musical,James and the Giant Peach,1996
1733,75,661,5,977851143,F,1,10,01748,Animation|Children's|Musical,James and the Giant Peach,1996
1734,78,661,4,977811895,F,45,1,98029,Animation|Children's|Musical,James and the Giant Peach,1996
1736,92,661,1,986187347,F,18,4,44243,Animation|Children's|Musical,James and the Giant Peach,1996


In [7]:
table2 = df2.pivot_table('Rating',\
index='name',aggfunc=np.mean)\
.sort_values(by='Rating',ascending = False).head(10)
table2

,Rating
name,
Bittersweet Motel,5.0
Coldblooded,5.0
Twice Upon a Yesterday,5.0
Message to Love: The Isle of Wight Festival,5.0
Ayn Rand: A Sense of Life,5.0
"Saltmen of Tibet, The",5.0
Belly,5.0
"Gate of Heavenly Peace, The",5.0
24 7: Twenty Four Seven,5.0


c)

In [8]:
table3 = df1.groupby(['name','MovieID'],as_index=False)\
['Rating'].agg('count')
table4 = table3[table3.Rating>=250]
table4.columns = ['name','MovieID','#Rating']
table4[:6]

,name,MovieID,#Rating
3,"'burbs, The",2072,303
6,10 Things I Hate About You,2572,700
7,101 Dalmatians,1367,364
8,101 Dalmatians,2085,565
9,12 Angry Men,1203,616
10,"13th Warrior, The",2826,750


In [9]:
table5 = df1.groupby('MovieID',as_index=False)\
['Rating'].agg('mean')
table5.columns = ['MovieID','aveRating']
table5[:6]

,MovieID,aveRating
0,1,4.146846
1,2,3.201141
2,3,3.016736
3,4,2.729412
4,5,3.006757
5,6,3.878723


In [10]:
table6 = pd.merge(table4,table5).\
sort_values(by='aveRating',ascending = False)
table6[['name','aveRating']].head(5)

,name,aveRating
959,Seven Samurai (The Magnificent Seven) (Shichin...,4.560510
971,"Shawshank Redemption, The",4.554558
460,"Godfather, The",4.524966
233,"Close Shave, A",4.520548
1151,"Usual Suspects, The",4.517106


d)

In [11]:
df3 = df1[df1.Gender=='F']
df4 = df1[df1.Gender=='M']

In [12]:
table7 = df3.groupby(['name','MovieID'],as_index=False)\
['Rating'].agg('mean')
table7.columns = ['name','MovieID','aveRatingF']
table8 = df4.groupby(['name','MovieID'],as_index=False)\
['Rating'].agg('mean')
table8.columns = ['name','MovieID','aveRatingM']

In [13]:
table9 = table7.merge(table8, how='inner', on='MovieID')
table9['diff'] = abs(table9['aveRatingF']-table9['aveRatingM'])
table9.sort_values(by='diff',ascending = False)
table9[['name_x','diff']].head(5)

,name_x,diff
0,"$1,000,000 Duck",0.613095
1,'Night Mother,0.035948
2,'Til There Was You,0.057658
3,"'burbs, The",0.168607
4,...And Justice for All,0.139547
